In [1]:
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn import datasets

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
boston = datasets.load_boston()
print(boston.data.shape, boston.target.shape)

(506, 13) (506,)


In [4]:
X_train = boston.data[:380]
y_train = boston.target[:380]
X_test = boston.data[380:]
y_test = boston.target[380:]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(380, 13) (380,) (126, 13) (126,)


In [5]:
mins = np.empty([0])
maxs = np.empty([0])
for i in range(X_train.shape[1]):
    mins = np.append(mins, (np.amin(X_train[:, i]))) 
    maxs = np.append(maxs, (np.amax(X_train[:, i])))
print(mins.shape, maxs.shape , X_train.shape[1])

(13,) (13,) 13


In [6]:
MAX = 10000000

In [7]:
def h(arr):
    return np.mean((arr - np.mean(arr))**2)

In [8]:
class Node:   
    def __init__(self, R_or_L, X, y):
        self.R_or_L = R_or_L
        self.X = X
        self.y = y
        self.feature = None
        self.bound = None
        self.mean = None
        self.right_neighbour = None
        self.left_neighbour = None
        
    def set_left_neighbour(self, Node):
        self.left_neighbour = Node
    def set_right_neighbour(self, Node):
        self.right_neighbour = Node
    def set_mean(self):
        self.mean = np.mean(self.y)
    def set_feature(self, feature):
        self.feature = feature
    def set_bound(self, bound):
        self.bound = bound

In [9]:
class DecisionTree:
    def __init__(self, depth = 10):
        self.depth = depth
    def partition(self, node, curdepth):
        node.set_mean()
        if(curdepth > self.depth):
            return
        if(node.X.shape[0] == 0 or node.X is None):
            return
        X_left = np.empty([0])
        y_left = np.empty([0])
        X_right = np.empty([0])
        y_right = np.empty([0])
        minG = MAX
        mfeature = 0
        mbound = 0
        for i in range(node.X.shape[1]):
            for j in range(node.X.shape[0]):
                X_rcur = node.X[node.X[:, i] <= node.X[j,i]]
                y_rcur = node.y[node.X[:, i] <= node.X[j,i]]
                X_lcur = node.X[node.X[:, i] > node.X[j,i]]
                y_lcur = node.y[node.X[:, i] > node.X[j,i]]
                if(X_rcur is None or X_rcur.shape[0] == 0 or X_lcur.shape[0] == 0 or X_lcur is None):
                    continue
                r = y_rcur.shape[0]
                l = y_lcur.shape[0]
                G = l/node.X.shape[0] * h(X_lcur) + r/node.X.shape[0] * h(X_rcur)
                if(minG > G):
                    minG = G
                    X_left = X_lcur
                    X_right = X_rcur
                    y_left = y_lcur
                    y_right = y_rcur
                    mfeature = i
                    mbound = j
        if(minG == MAX):
            return
        if(X_left is not None and X_left.shape[0] != 0 and X_right is not None and X_right.shape[0] != 0):
            node.set_feature(mfeature)
            node.set_bound(mbound)
            leftnode = Node(1, X_left, y_left)
            node.set_left_neighbour(leftnode)
            self.partition(node.left_neighbour, curdepth + 1)
            rightnode = Node(0, X_right, y_right)
            node.set_right_neighbour(rightnode)
            self.partition(node.right_neighbour, curdepth + 1)
    def fit(self, X_train, y_train):
        self.root = Node(-1, X_train, y_train)
        self.partition(self.root, 1)
    def prediction(self, node, arr):
        if(node.feature is None):
            return(node.mean)
        if(arr[node.feature] <= node.bound):
            return(self.prediction(node.right_neighbour, arr))
        else:
            return(self.prediction(node.left_neighbour, arr))
    def predict(self, X_test, y_test):
        y_pred = np.empty([0])
        for i in range(X_test.shape[0]):
            y_pred = np.append(y_pred, self.prediction(self.root, X_test[i]))
        return y_pred
    def mse_check(self, y_pred, y_test):
        return(np.mean((y_test - y_pred)**2))
    

In [10]:
tree = DecisionTree(depth=6)
tree.fit(X_train, y_train)

In [11]:
y_pred = tree.predict(X_test, y_test)

In [12]:
print(tree.mse_check(y_pred, y_test))

28.6576112496
